# DNN with Hyper-Parameters Tuning

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # disable GPU
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout

#hyper-parameter tuning imports
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time
LOG_DIR = f"{int(time.time())}" # folder names as a timestamp

SEED = 111 # constant seed for reproducibility
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

df = pd.read_csv("UFC_TRAIN.csv")

# tackling imbalance issue
theMin = df["Winner"].value_counts().min()
minority = df[df["Winner"]==1].iloc[0:theMin]
undersampleMaj = df[df["Winner"]==0].iloc[0:theMin]
df = pd.concat([minority, undersampleMaj], axis=0)
print(df["Winner"].value_counts())

# train/test split
X = df.drop(["date","Winner","B_fighter","R_fighter"], axis=1).values
y = df["Winner"].values
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=SEED)

# scaling
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(f"X_train shape: {X_train.shape} | X_test shape: {X_test.shape} | y_train shape: {y_train.shape} | y_test shape: {y_test.shape}")

# Parameters
MIN = 30
MAX = 256
STEP = 16
MAX_TRIALS = 2
EXE_PER_TRIAL = 1
EPOCHS = 10
PATIENCE = 16

# function to build the model (argument: hyper-parameter)
def build_model(hp):
    # model
    model = Sequential()
    
    # first layer's no. of neurons = hp.Int range of values to test
    model.add(Dense(hp.Int("input_units", min_value=MIN, max_value=MAX, step=STEP), activation='relu'))
    
    # range of 1 to 10 layers to test
    for i in range(hp.Int("no. Of Hidden Layers", 1, 5)):
        # for each added layer, again test range of neurons and add dropout
        model.add(Dense(hp.Int(f"Hidden_layer_{i+1}_units", min_value=MIN, max_value=MAX, step=STEP), activation='relu'))
        model.add(Dropout(0.5))
    
    # output layer
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

tuner = RandomSearch(
        build_model, # name of the function that builds the model
        objective="val_accuracy", # the thing that we're interested to trace
        max_trials = MAX_TRIALS, # no. of combinations to try
        executions_per_trial = EXE_PER_TRIAL, # no. of times to train each combination (true avg)
        directory=LOG_DIR) # directory to save the outputs

# prevent divergence of loss & val_loss via early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=PATIENCE)

1    1336
0    1336
Name: Winner, dtype: int64
X_train shape: (2137, 42) | X_test shape: (535, 42) | y_train shape: (2137,) | y_test shape: (535,)


In [2]:
tuner.search(x=X_train,
             y=y_train,
             epochs = EPOCHS,
#             batch_size = 64,
             callbacks=[early_stop],
             validation_data=(X_test,y_test))

Train on 2137 samples, validate on 535 samples
Epoch 1/10
2137/2137 [==============================] - ETA: 2:23 - loss: 0.6937 - accuracy: 0.56 - ETA: 22s - loss: 0.7000 - accuracy: 0.5052 - ETA: 17s - loss: 0.7071 - accuracy: 0.484 - ETA: 13s - loss: 0.7027 - accuracy: 0.490 - ETA: 10s - loss: 0.7063 - accuracy: 0.483 - ETA: 7s - loss: 0.7037 - accuracy: 0.486 - ETA: 5s - loss: 0.7001 - accuracy: 0.51 - ETA: 4s - loss: 0.6980 - accuracy: 0.50 - ETA: 3s - loss: 0.6977 - accuracy: 0.49 - ETA: 2s - loss: 0.6967 - accuracy: 0.49 - ETA: 2s - loss: 0.6966 - accuracy: 0.50 - ETA: 2s - loss: 0.6978 - accuracy: 0.49 - ETA: 2s - loss: 0.6983 - accuracy: 0.48 - ETA: 1s - loss: 0.6977 - accuracy: 0.49 - ETA: 1s - loss: 0.6968 - accuracy: 0.49 - ETA: 1s - loss: 0.6958 - accuracy: 0.50 - ETA: 1s - loss: 0.6954 - accuracy: 0.50 - ETA: 0s - loss: 0.6959 - accuracy: 0.50 - ETA: 0s - loss: 0.6955 - accuracy: 0.50 - ETA: 0s - loss: 0.6951 - accuracy: 0.50 - ETA: 0s - loss: 0.6956 - accuracy: 0.50 - 4s 

Train on 2137 samples, validate on 535 samples
Epoch 1/10
2137/2137 [==============================] - ETA: 1:43 - loss: 0.6915 - accuracy: 0.65 - ETA: 20s - loss: 0.6862 - accuracy: 0.6062 - ETA: 6s - loss: 0.6943 - accuracy: 0.535 - ETA: 3s - loss: 0.6938 - accuracy: 0.52 - ETA: 2s - loss: 0.6951 - accuracy: 0.52 - ETA: 1s - loss: 0.6961 - accuracy: 0.52 - ETA: 0s - loss: 0.6956 - accuracy: 0.51 - ETA: 0s - loss: 0.6944 - accuracy: 0.51 - 2s 1ms/sample - loss: 0.6941 - accuracy: 0.5194 - val_loss: 0.6859 - val_accuracy: 0.5832
Epoch 2/10
2137/2137 [==============================] - ETA: 0s - loss: 0.7032 - accuracy: 0.46 - ETA: 0s - loss: 0.6914 - accuracy: 0.54 - ETA: 0s - loss: 0.6963 - accuracy: 0.50 - ETA: 0s - loss: 0.6954 - accuracy: 0.50 - ETA: 0s - loss: 0.6934 - accuracy: 0.52 - ETA: 0s - loss: 0.6919 - accuracy: 0.53 - ETA: 0s - loss: 0.6899 - accuracy: 0.53 - ETA: 0s - loss: 0.6906 - accuracy: 0.53 - 1s 265us/sample - loss: 0.6900 - accuracy: 0.5386 - val_loss: 0.6810 - va

INFO:tensorflow:Oracle triggered exit


In [3]:
print(tuner.results_summary()) # top 10 trials
print(tuner.get_best_hyperparameters()[0].values) #values of best hyper-parameters
#print(tuner.get_best_models()[0].summary())

None
{'input_units': 30, 'no. Of Hidden Layers': 1, 'Hidden_layer_1_units': 190, 'Hidden_layer_2_units': 222, 'Hidden_layer_3_units': 62, 'Hidden_layer_4_units': 222}
